In [ ]:
%cd /Users/yao/Desktop/TrueMan/big3/資管專題

In [ ]:
import json
import requests
import lxml
import time
from io import StringIO, BytesIO
import lxml.etree
import re
import sys

In [ ]:
def parse_content(htmlin):
    '''
        input : str, raw html code
        output: dict() , post_dictionary of posts' content and their link
    '''
    parser = lxml.etree.HTMLParser()
    html = lxml.etree.parse(BytesIO(htmlin.encode('utf-8')), parser) 
    pdict_tmp = html.xpath("/html/body/script[@type='text/javascript']")[0].text[21:-1]
    pdict_tmp = re.sub(r"\bfalse\b","False",pdict_tmp)  #is equal to '\\bfalse\\b'
    pdict_tmp = re.sub(r"\btrue\b","True",pdict_tmp)
    pdict_tmp = re.sub(r"\bnull\b","None",pdict_tmp)
    pdict_tmp = eval(pdict_tmp)
    return pdict_tmp

In [ ]:
def get_html_text(URL,rs=requests.session()):
    '''
        input : str, URL wanted to crawl
        output: str, text of html code
    '''
    html1 = rs.get(URL)
    html1.encoding = 'utf-8'
    return html1.text

In [ ]:
def get_usrName_and_cmtList(tmp_code):
    '''
        input:  str, post_id's code, ex:"BE3l0CKOvTr".
        output: tuple, (str,list), (this user's name,[comments on this posts]).
    '''
    getURL = "https://www.instagram.com/p/%s/"%(tmp_code)
    result = get_html_text(getURL)
    
    usr_name = re.findall("See this Instagram photo by @([a-zA-Z0-9_\.]*)",result)
    if len(usr_name) == 0:
        usr_name = re.findall("See this Instagram video by @([a-zA-Z0-9_\.]*)",result)
    else:
        usr_name = usr_name[0]
        
    page_content = parse_content(result)
    cmt_list = []
    for cmt in  page_content['entry_data']['PostPage'][0]['media']['comments']['nodes']:
        cmt_list.append(cmt['text'])
    return (usr_name,cmt_list)

In [ ]:
def add_recent_posts(person,days=3):
    '''
        input : dict, this tag_post's dict.
        output: dict, this tag_post's dict + it's recent posts.
    '''
    getURL = "https://www.instagram.com/%s/"%(person['username'])
    pdict_tmp = parse_content(get_html_text(getURL))
    page_tmp = pdict_tmp['entry_data']['ProfilePage'][0]['user']['media']['page_info']
    over_7_days = False

    for post in pdict_tmp['entry_data']['ProfilePage'][0]['user']['media']['nodes']:
        if abs(post_time-post['date']) < 86400*days:  #7 days
            (usrName,cmtList) = get_usrName_and_cmtList(post['code'])
            post['comments']['contents'] = cmtList
            post['username'] = usrName
            if 'recent_posts' not in person:
                person['recent_posts'] = [post]
            else:
                person['recent_posts'].append(post)

    while page_tmp['has_next_page'] == True:
        getURL = "https://www.instagram.com/%s/?max_id=%s"%(person['username'],page_tmp['end_cursor'])
        pdict_tmp = parse_content(get_html_text(getURL))
        page_tmp = pdict_tmp['entry_data']['ProfilePage'][0]['user']['media']['page_info']
        over_7_days = False
        for post in pdict_tmp['entry_data']['ProfilePage'][0]['user']['media']['nodes']:
            if abs(post_time-post['date']) < 86400*days:  #7 days
                (usrName,cmtList) = get_usrName_and_cmtList(post['code'])
                post['comments']['contents'] = cmtList
                post['username'] = usrName
                if 'recent_posts' not in person:
                    person['recent_posts'] = [post]
                else:
                    person['recent_posts'].append(post)
            if post_time-post['date'] > 86400*days:
                over_7_days = True
                break
        if over_7_days==True:
            break
    return person

In [ ]:
searchTag = "馬子狗"

In [ ]:
getURL = "https://www.instagram.com/explore/tags/%s/"%(searchTag)
rs = requests.session()
result = get_html_text(getURL,rs)

In [ ]:
p_list = []   #all the post list
pdict_tmp = parse_content(result)
p_list.extend(pdict_tmp['entry_data']['TagPage'][0]['tag']['media']['nodes'])
page_tmp = pdict_tmp['entry_data']['TagPage'][0]['tag']['media']['page_info']
print("posts has crawl :")
while page_tmp['has_next_page'] == True:
#     time.sleep(0.1)
    getURL = "https://www.instagram.com/explore/tags/%s/?max_id=%s"%(searchTag,page_tmp['end_cursor'])
    pdict_tmp = parse_content(get_html_text(getURL,rs))
    p_list.extend(pdict_tmp['entry_data']['TagPage'][0]['tag']['media']['nodes'])
    page_tmp = pdict_tmp['entry_data']['TagPage'][0]['tag']['media']['page_info']
    sys.stdout.write("%d,"%(len(p_list)))

In [ ]:
print("add near_posts for %d posts:"%(len(p_list)))
for i in range(len(p_list)):
    tag_post_id = p_list[i]['code']
    (usrName,cmtList) = get_usrName_and_cmtList(tag_post_id) #BE5QcuKNI9P
    p_list[i]['comments']['contents'] = cmtList
    p_list[i]['username'] = usrName
    post_time = p_list[i]['date']
    p_list[i] = add_recent_posts(p_list[i],3)
    if i %10 == 0 :
        sys.stdout.write("%d,"%(i))

In [ ]:
p_final = []
for post in p_list:
    p_tmp = {'main_post':{},'near_posts':[]}
    p_tmp['main_post']['content'] = post['caption']
    p_tmp['main_post']['comments'] = post['comments']['contents']
    p_tmp['main_post']['date'] = post['date']
    for npost in post['recent_posts']:
        npost_dic = {}
        if npost['date'] != p_tmp['main_post']['date'] :
            try:
                npost_dic['content'] = npost['caption']
            except:
                npost_dic['content'] = []
            npost_dic['date'] = npost['date']
            npost_dic['comments'] = npost['comments']['contents']
            p_tmp['near_posts'].append(npost_dic)
    p_final.append(p_tmp)

In [ ]:
with open("IG_%s.txt"%(searchTag),'w') as f1:
    f1.write(str(p_final))

In [ ]:
# read usage:
# with open("IG_%s.txt"%(searchTag),'r') as f1:
#     list_input = eval(f1.read())